In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

# 1. Initializing Model from Hugging Face

In [5]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

C:\Users\chana\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\chana\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


# 2. Encode and Calculate Sentiment

In [139]:
tokens = tokenizer.encode('I dislike this', return_tensors='pt')

In [140]:
result = model(tokens)

In [141]:
result.logits

tensor([[ 2.1701,  2.0569,  0.7942, -1.7036, -2.6528]],
       grad_fn=<AddmmBackward0>)

In [142]:
int(torch.argmax(result.logits))+1

1

# 3. Collect Reviews

In [99]:
import requests
from bs4 import BeautifulSoup

# Getting Access to Website
url = "https://www.flipkart.com/ephemeral-lcd-writing-8-5-inch-tablet-electronic-drawing-doodle-board-green/product-reviews/itm0272268d11942?pid=ETYGF9XNDZJ5KRZZ&lid=LSTETYGF9XNDZJ5KRZZIUJ5H4&marketplace=FLIPKART"
response = requests.get(url)

# Scraping Data
soup = BeautifulSoup(response.text, "html.parser")
review_elements = soup.find_all("div", class_="t-ZTKy")
reviews = [element.get_text(strip=True)[:-9] for element in review_elements]

# Display
for review in reviews:
    print(review)


Such a amazing  tablet on lower price
Good
Go for it
Amazing product smooth writing 👌🏻😊😊
Nice product
Very nice 👍👍
For baby it's good and  make fun ND njoy with this to draw something  or writing purpose but you can't delete one particular word .You have to delete everything at once which they gave.One more thing after deleting it takes time to draw  again something  .
Very nice product 👍
Best writing experience😜😜 Amazing
Nice 👍


# 4. Load Reviews into DataFrame and Score

In [100]:
import numpy as np
import pandas as pd

In [101]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [102]:
df['review'].iloc[0]

'Such a amazing  tablet on lower price'

In [103]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [104]:
sentiment_score(df['review'].iloc[1])

4

In [105]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [106]:
df

,review,sentiment
0,Such a amazing tablet on lower price,5
1,Good,4
2,Go for it,5
3,Amazing product smooth writing 👌🏻😊😊,5
4,Nice product,4
5,Very nice 👍👍,5
6,For baby it's good and make fun ND njoy with ...,3
7,Very nice product 👍,5
8,Best writing experience😜😜 Amazing,5
9,Nice 👍,5


In [110]:
df['review'].iloc[6]

"For baby it's good and  make fun ND njoy with this to draw something  or writing purpose but you can't delete one particular word .You have to delete everything at once which they gave.One more thing after deleting it takes time to draw  again something  ."